In [1]:
# Installing and importing libraries

In [2]:
#%pip install transformers
%pip install emoji
%pip install contractions
%pip install tensorflow
#%pip install --upgrade tensorflow transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.2 MB/s eta 0:00:00


In [6]:
%pip install h5py
%pip install typing-extensions
%pip install wheel

In [4]:
%pip uninstall -q transformers -y

In [7]:
%pip install -q transformers==4.17

In [9]:
# Data manipulation libraries
import sys, os
import pandas as pd
import numpy as np
import json

import emoji
import contractions
import re

# Scikit-learn packages
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

# Packages to define a BERT model
from transformers import TFBertModel, BertTokenizerFast, BertConfig

# Keras and TensorFlow packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Loading dataset
# Importing train, validation and test datasets with preprocessed texts and labels
train_data = pd.read_csv("./drive/MyDrive/ColabNotebooks/semeval_project/train_data.csv")
val_data = pd.read_csv("./drive/MyDrive/ColabNotebooks/semeval_project/val_data.csv")
test_data = pd.read_csv("./drive/MyDrive/ColabNotebooks/semeval_project/test_data.csv")

# Shape validation
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

# Defining the emotion columns
emotion = ['anger', 'fear', 'joy', 'sadness', 'surprise']

(2214, 6)
(276, 7)
(278, 7)


In [ ]:
# Modeling : BERT (Bidirectional Encoder Representations from Transformers)

In [12]:
# Computing max length of samples
full_text = pd.concat([train_data['clean_text'], val_data['clean_text'], test_data['clean_text']])
max_length = full_text.apply(lambda x: len(x.split())).max()
max_length

86

In [8]:
import tensorflow as tf
import transformers

print(tf.__version__)  # Ensure >=2.6
print(transformers.__version__)  # Ensure >=4.12

2.17.1
4.17.0


In [13]:
# Importing BERT pre-trained model and tokenizer
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name, output_hidden_states=False)
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:1383: UserWarning: Layer 'tf_bert_model' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''The following keyword arguments are not supported by this model: ['kwargs'].''
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'tf_bert_model', however the layer does not have a `build()` method implemented and it looks like 

AttributeError: Exception encountered when calling TFBertMainLayer.call().

[1m'NoneType' object has no attribute 'shape'[0m

Arguments received by TFBertMainLayer.call():
  • input_ids=tf.Tensor(shape=(3, 5), dtype=int32)
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=True
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False
  • kwargs=<class 'inspect._empty'>

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "I am happy today"
inputs = tokenizer(text, return_tensors="tf", padding="max_length", truncation=True, max_length=86)

In [14]:
# function for creating BERT based model
def create_model(nb_labels):

  # Load the MainLayer
  bert = transformer_model.layers[0]

  # Build the model inputs
  input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
  attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32')
  token_ids = Input(shape=(max_length,), name='token_ids', dtype='int32')
  inputs = {'input_ids': input_ids, 'attention_mask': attention_mask, 'token_ids': token_ids}

  # Load the Transformers BERT model as a layer in a Keras model
  bert_model = bert(inputs)[1]
  dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
  pooled_output = dropout(bert_model, training=False)

  # Then build the model output
  emotion = Dense(units=nb_labels, activation="sigmoid", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(pooled_output)
  outputs = emotion

  # And combine it all in a model object
  model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel')

  return model



In [15]:
model = create_model(5)
model.summary()

NameError: name 'transformer_model' is not defined

2.18.0
4.47.1


In [ ]:
from transformers import BertTokenizer
import tensorflow as tf

# Tokenize the input text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = ["Sample text for BERT."]
inputs = tokenizer(text, return_tensors="tf", padding="max_length", truncation=True, max_length=86)

# Ensure inputs are TensorFlow tensors
print(type(inputs['input_ids']))  # Should be <class 'tensorflow.python.framework.ops.EagerTensor'>
print(type(inputs['attention_mask']))  # Should be <class 'tensorflow.python.framework.ops.EagerTensor'>

# Pass these inputs to the model

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
#nb_labels = ['anger', 'fear', 'joy', 'sadness', 'surprise']

In [ ]:
tf.debugging.disable_traceback_filtering()

In [ ]:
model = create_model(nb_labels=5)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_7' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_7' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 86), dtype=float32, sparse=False, name=keras_tensor_14>
  • attention_mask=<KerasTensor shape=(None, 86), dtype=float32, sparse=False, name=keras_tensor_17>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [ ]:
# Definition of the model architecture

In [ ]:
from transformers import TFBertModel, BertConfig
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

def create_model(nb_labels):
    # Load BERT model and configuration
    config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=False)
    bert = TFBertModel.from_pretrained('bert-base-uncased', config=config)

    # Define input layers
    input_ids = Input(shape=(86,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(86,), dtype=tf.int32, name="attention_mask")

    # Add a Lambda layer to cast the KerasTensor to TensorFlow Tensor
    def convert_to_tensor(inputs):
        return tf.identity(inputs)

    input_ids_tf = tf.keras.layers.Lambda(convert_to_tensor)(input_ids)
    attention_mask_tf = tf.keras.layers.Lambda(convert_to_tensor)(attention_mask)

    # Pass the inputs to the BERT model
    bert_outputs = bert(input_ids=input_ids_tf, attention_mask=attention_mask_tf)
    pooled_output = bert_outputs.pooler_output  # Use [CLS] token representation

    # Add a Dropout and Dense layer for classification
    dropout = Dropout(config.hidden_dropout_prob)(pooled_output)
    output = Dense(nb_labels, activation='softmax')(dropout)

    # Create the Model
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    return model


In [ ]:
# Creating a model instance
#model = create_model(nb_labels=5)

# Compile the model before summarizing
#model.compile(optimizer=Adam(learning_rate=2e-5),
             # loss='binary_crossentropy',
             # metrics=['accuracy'])

# Take a look at the model
#model.summary()


#OR
# Creating a model instance
model = create_model(5) # Assuming 5 labels based on your previous code

# Compile the model before summarizing
model.compile(optimizer=Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_8' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_8' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 86), dtype=float32, sparse=False, name=keras_tensor_20>
  • attention_mask=<KerasTensor shape=(None, 86), dtype=float32, sparse=False, name=keras_tensor_23>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [ ]:
# Creating train, validation and test variables
X_train = train_data['clean_text']
y_train = train_data.loc[:, emotion].values.astype(float)

X_val = val_data['clean_text']
y_val = val_data.loc[:, emotion].values.astype(float)

X_test = test_data['clean_text']
y_test = test_data.loc[:, emotion].values.astype(float)

# Tokenizing train data
train_token = tokenizer(
    text = X_train.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing valisation data
val_token = tokenizer(
    text = X_val.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing test data
test_token = tokenizer(
    text = X_test.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length',
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [ ]:
# Creating BERT compatible inputs with Input Ids, attention masks and token Ids
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_ids': train_token['token_type_ids']}
val = {'input_ids': val_token['input_ids'], 'attention_mask': val_token['attention_mask'],'token_ids': val_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_ids': test_token['token_type_ids']}

In [ ]:
# Creating TF tensors
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(16)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_val)).shuffle(len(val)).batch(16)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(16)

In [ ]:
# Class weights for multi-label and custom loss function

In [ ]:
# Function for calculating multilabel class weights
def calculating_class_weights(y_true):
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
      # Changed to use keyword arguments instead of positional arguments
        weights[i] = compute_class_weight(class_weight='balanced', classes=np.array([0.,1.]), y=y_true[:, i])
    return weights

class_weights = calculating_class_weights(y_train)

In [ ]:
 # Custom loss function for multilabel
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

In [ ]:
# Model Training

In [ ]:

# Set an optimizer
optimizer = Adam(
    learning_rate=5.e-05,
    )

# Set loss
loss = get_weighted_loss(class_weights)

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss)

# train the model
history = model.fit(train_tensor,
                    epochs=4,
                    validation_data=val_tensor,
                    )

NameError: name 'model' is not defined